<a href="https://colab.research.google.com/github/AnovaYoung/AI-System-for-Image-Restoration-and-Enhancement/blob/Modeling/Real_ESRGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

zip_path = "/content/drive/My Drive/super_resolution_resized_split.zip"  # Adjust this path if necessary
extract_path = "/content/super_resolution_dataset"

print("Extracting dataset...")
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracting dataset...
Extraction complete!


# DOWNSAMPLE DATASET



In [4]:
import os
import random
import shutil

# Original directories
original_dirs = {
    "train_lr": "/content/super_resolution_dataset/train/super_resolution_lr",
    "train_hr": "/content/super_resolution_dataset/train/super_resolution_hr",
    "val_lr": "/content/super_resolution_dataset/val/super_resolution_lr",
    "val_hr": "/content/super_resolution_dataset/val/super_resolution_hr",
    "test_lr": "/content/super_resolution_dataset/test/super_resolution_lr",
    "test_hr": "/content/super_resolution_dataset/test/super_resolution_hr",
}

# Reduced dataset directories
reduced_base_dir = "/content/super_resolution_dataset/reduced"
reduced_dirs = {
    "train_lr": os.path.join(reduced_base_dir, "train/super_resolution_lr"),
    "train_hr": os.path.join(reduced_base_dir, "train/super_resolution_hr"),
    "val_lr": os.path.join(reduced_base_dir, "val/super_resolution_lr"),
    "val_hr": os.path.join(reduced_base_dir, "val/super_resolution_hr"),
    "test_lr": os.path.join(reduced_base_dir, "test/super_resolution_lr"),
    "test_hr": os.path.join(reduced_base_dir, "test/super_resolution_hr"),
}

# Create reduced directories
for path in reduced_dirs.values():
    os.makedirs(path, exist_ok=True)

# Sampling sizes
sample_sizes = {
    "train": 15000,  # Train size
    "val": 3000,     # Validation size
    "test": 2000,    # Test size
}

# Function to sample and copy files
def sample_and_copy(original_lr, original_hr, reduced_lr, reduced_hr, num_samples):
    # Get sorted file lists
    lr_files = sorted(os.listdir(original_lr))
    hr_files = sorted(os.listdir(original_hr))

    # Ensure the LR and HR file lists match
    assert len(lr_files) == len(hr_files), "Mismatch between LR and HR files!"

    # Randomly sample file indices
    sampled_indices = random.sample(range(len(lr_files)), num_samples)

    # Copy selected files to reduced directories
    for idx in sampled_indices:
        shutil.copy(os.path.join(original_lr, lr_files[idx]), reduced_lr)
        shutil.copy(os.path.join(original_hr, hr_files[idx]), reduced_hr)

# Apply sampling for train, val, and test sets
sample_and_copy(
    original_dirs["train_lr"], original_dirs["train_hr"],
    reduced_dirs["train_lr"], reduced_dirs["train_hr"],
    sample_sizes["train"]
)

sample_and_copy(
    original_dirs["val_lr"], original_dirs["val_hr"],
    reduced_dirs["val_lr"], reduced_dirs["val_hr"],
    sample_sizes["val"]
)

sample_and_copy(
    original_dirs["test_lr"], original_dirs["test_hr"],
    reduced_dirs["test_lr"], reduced_dirs["test_hr"],
    sample_sizes["test"]
)

print("Reduced dataset created successfully!")


Reduced dataset created successfully!


In [5]:
import shutil

# Directories to delete
dirs_to_delete = [
    "/content/super_resolution_dataset/test",
    "/content/super_resolution_dataset/train",
    "/content/super_resolution_dataset/val",
    "/content/sample_data",

]

# Delete directories
for dir_path in dirs_to_delete:
    try:
        shutil.rmtree(dir_path)
        print(f"Deleted: {dir_path}")
    except FileNotFoundError:
        print(f"Directory not found, skipping: {dir_path}")
    except Exception as e:
        print(f"Error deleting {dir_path}: {e}")

print("Selected directories deleted successfully.")

Deleted: /content/super_resolution_dataset/test
Deleted: /content/super_resolution_dataset/train
Deleted: /content/super_resolution_dataset/val
Deleted: /content/sample_data
Selected directories deleted successfully.


In [6]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

# Directories for reduced dataset
reduced_dirs = {
    "train_lr": "/content/super_resolution_dataset/reduced/train/super_resolution_lr",
    "train_hr": "/content/super_resolution_dataset/reduced/train/super_resolution_hr",
    "val_lr": "/content/super_resolution_dataset/reduced/val/super_resolution_lr",
    "val_hr": "/content/super_resolution_dataset/reduced/val/super_resolution_hr",
    "test_lr": "/content/super_resolution_dataset/reduced/test/super_resolution_lr",
    "test_hr": "/content/super_resolution_dataset/reduced/test/super_resolution_hr",
}

# Output directories for normalized .npy files
normalized_base_dir = "/content/super_resolution_dataset/normalized_reduced"
normalized_dirs = {key: path.replace("reduced", "normalized_reduced") for key, path in reduced_dirs.items()}

# Create directories for normalized dataset
for path in normalized_dirs.values():
    os.makedirs(path, exist_ok=True)

# Normalize images and save as .npy
def normalize_and_save(input_dir, output_dir):
    for img_name in tqdm(os.listdir(input_dir), desc=f"Normalizing {input_dir}"):
        input_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name.replace(".png", ".npy"))

        with Image.open(input_path) as img:
            img_array = np.asarray(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
            img_array = img_array * 2 - 1  # Normalize to [-1, 1]
            np.save(output_path, img_array)  # Save as .npy

# Apply normalization to all splits
for key, input_dir in reduced_dirs.items():
    normalize_and_save(input_dir, normalized_dirs[key])

print("Reduced dataset normalized and saved as .npy!")


Normalizing /content/super_resolution_dataset/reduced/train/super_resolution_lr: 100%|██████████| 15000/15000 [00:19<00:00, 778.92it/s]
Normalizing /content/super_resolution_dataset/reduced/train/super_resolution_hr: 100%|██████████| 15000/15000 [03:17<00:00, 76.10it/s]
Normalizing /content/super_resolution_dataset/reduced/val/super_resolution_lr: 100%|██████████| 3000/3000 [00:06<00:00, 476.75it/s]
Normalizing /content/super_resolution_dataset/reduced/val/super_resolution_hr: 100%|██████████| 3000/3000 [00:44<00:00, 67.31it/s]
Normalizing /content/super_resolution_dataset/reduced/test/super_resolution_lr: 100%|██████████| 2000/2000 [00:04<00:00, 430.44it/s]
Normalizing /content/super_resolution_dataset/reduced/test/super_resolution_hr: 100%|██████████| 2000/2000 [00:29<00:00, 68.02it/s]

Reduced dataset normalized and saved as .npy!


In [ ]:
!pip install basicsr
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url

# Initialize the Real-ESRGAN model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth'  # URL of the pre-trained model

# Download the model
cached_file = load_file_from_url(
    url=model_path, model_dir='/content/realesrgan_models', progress=True, file_name=None
)

model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(cached_file)['params_ema'], strict=True)  # Use 'params_ema' for better results
model.eval()
model = model.to(device)
print("Model loaded successfully!")

# SET UP DATALOADER

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np

# Define custom dataset for normalized .npy files
class SuperResolutionNpyDataset(Dataset):
    def __init__(self, lr_dir, hr_dir):
        self.lr_dir = lr_dir
        self.hr_dir = hr_dir
        self.lr_files = sorted(os.listdir(lr_dir))
        self.hr_files = sorted(os.listdir(hr_dir))

    def __len__(self):
        return len(self.lr_files)

    def __getitem__(self, idx):
        lr_path = os.path.join(self.lr_dir, self.lr_files[idx])
        hr_path = os.path.join(self.hr_dir, self.hr_files[idx])

        # Load normalized .npy arrays
        lr_image = np.load(lr_path)
        hr_image = np.load(hr_path)

        # Convert to PyTorch tensors
        lr_tensor = torch.tensor(lr_image).permute(2, 0, 1).float()  # HWC to CHW
        hr_tensor = torch.tensor(hr_image).permute(2, 0, 1).float()

        return lr_tensor, hr_tensor

# Directories for normalized .npy files
train_lr_dir = "/content/super_resolution_dataset/normalized_reduced/train/super_resolution_lr"
train_hr_dir = "/content/super_resolution_dataset/normalized_reduced/train/super_resolution_hr"
val_lr_dir = "/content/super_resolution_dataset/normalized_reduced/val/super_resolution_lr"
val_hr_dir = "/content/super_resolution_dataset/normalized_reduced/val/super_resolution_hr"
test_lr_dir = "/content/super_resolution_dataset/normalized_reduced/test/super_resolution_lr"
test_hr_dir = "/content/super_resolution_dataset/normalized_reduced/test/super_resolution_hr"

# Create datasets and dataloaders
batch_size = 16  # Adjust batch size if needed
train_dataset = SuperResolutionNpyDataset(train_lr_dir, train_hr_dir)
val_dataset = SuperResolutionNpyDataset(val_lr_dir, val_hr_dir)
test_dataset = SuperResolutionNpyDataset(test_lr_dir, test_hr_dir)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Verify dataloader
for lr_batch, hr_batch in train_loader:
    print(f"LR Batch Shape: {lr_batch.shape}, Range: {lr_batch.min().item()} to {lr_batch.max().item()}")
    print(f"HR Batch Shape: {hr_batch.shape}, Range: {hr_batch.min().item()} to {hr_batch.max().item()}")
    break


LR Batch Shape: torch.Size([16, 3, 128, 128]), Range: -1.0 to 1.0
HR Batch Shape: torch.Size([16, 3, 512, 512]), Range: -1.0 to 1.0


In [14]:
dummy_input = torch.randn(1, 3, 64, 64).to(device)  # Simulated LR image
with torch.no_grad():
    output = model(dummy_input)
print(f"Output Shape: {output.shape}")


Output Shape: torch.Size([1, 3, 256, 256])


In [17]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [18]:
import torch
torch.cuda.empty_cache()  # Clears cached memory


In [19]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'  # Avoid splitting large memory blocks


In [20]:
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast  # Import for mixed precision training

# Define loss function and optimizer
criterion = nn.L1Loss()  # Pixel-wise loss
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Adjust learning rate as needed
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # Optional learning rate scheduler
scaler = GradScaler()  # Mixed precision scaler

# Training parameters
num_epochs = 20
patience = 4  # Early stopping patience
best_val_loss = float('inf')
early_stop_counter = 0
accumulation_steps = 2  # Gradient accumulation steps
batch_size = 4  # Adjust if memory issues persist

# TensorBoard for logging
writer = SummaryWriter(log_dir="/content/runs/reduced_dataset_training")

# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0
    total_batches = len(train_loader)

    # Train
    for i, (lr_batch, hr_batch) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")):
        lr_batch, hr_batch = lr_batch.to(device), hr_batch.to(device)

        # Mixed precision training
        optimizer.zero_grad()
        with autocast():  # Forward pass in mixed precision
            sr_batch = model(lr_batch)
            loss = criterion(sr_batch, hr_batch)

        # Backward pass with scaled loss
        loss = loss / accumulation_steps
        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.step(optimizer)  # Scaled optimizer step
            scaler.update()  # Update the scaler
            optimizer.zero_grad()

        epoch_loss += loss.item() * accumulation_steps  # Accumulated loss

        # Log training progress
        writer.add_scalar("Batch Loss/Train", loss.item() * accumulation_steps, epoch * total_batches + i)

    train_loss = epoch_loss / len(train_loader)
    writer.add_scalar("Loss/Train", train_loss, epoch + 1)
    print(f"Epoch {epoch+1} Training Loss: {train_loss:.6f}")

    # Validation
    model.eval()
    val_loss = 0.0
    total_val_batches = len(val_loader)
    with torch.no_grad():
        for i, (lr_batch, hr_batch) in enumerate(tqdm(val_loader, desc=f"Validation {epoch+1}/{num_epochs}")):
            lr_batch, hr_batch = lr_batch.to(device), hr_batch.to(device)

            with autocast():  # Mixed precision during validation
                sr_batch = model(lr_batch)
                loss = criterion(sr_batch, hr_batch)

            val_loss += loss.item()

            # Log validation progress
            writer.add_scalar("Batch Loss/Validation", loss.item(), epoch * total_val_batches + i)

    val_loss /= len(val_loader)
    writer.add_scalar("Loss/Validation", val_loss, epoch + 1)
    print(f"Epoch {epoch+1} Validation Loss: {val_loss:.6f}")

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        # Save best model
        best_model_path = f"/content/drive/My Drive/best_model_epoch_{epoch+1}.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"Saved best model at epoch {epoch+1} to {best_model_path}")
    else:
        early_stop_counter += 1

    if early_stop_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break

    scheduler.step()  # Adjust learning rate

writer.close()
print("Training completed.")


<ipython-input-22-6eb08c16e983>:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Mixed precision scaler
Epoch 1/20:   0%|          | 0/938 [00:00<?, ?it/s]<ipython-input-22-6eb08c16e983>:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Forward pass in mixed precision
Epoch 1/20: 100%|██████████| 938/938 [11:56<00:00,  1.31it/s]


Epoch 1 Training Loss: 0.337367


Validation 1/20:   0%|          | 0/188 [00:00<?, ?it/s]<ipython-input-22-6eb08c16e983>:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision during validation
Validation 1/20: 100%|██████████| 188/188 [01:32<00:00,  2.02it/s]


Epoch 1 Validation Loss: 0.335613
Saved best model at epoch 1 to /content/drive/My Drive/best_model_epoch_1.pth


Epoch 2/20: 100%|██████████| 938/938 [11:13<00:00,  1.39it/s]


...

Epoch 18/20: 100%|██████████| 938/938 [09:50<00:00,  1.59it/s]


Epoch 18 Training Loss: 0.308292


Validation 18/20: 100%|██████████| 188/188 [00:48<00:00,  3.85it/s]


Epoch 18 Validation Loss: 0.316802


Epoch 19/20:  13%|█▎        | 125/938 [01:18<08:29,  1.59it/s]